In [ ]:
import json
import sys
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from folium.plugins import HeatMap
import folium as fl
from datetime import datetime

sns.set(style="whitegrid")
%matplotlib inline

In [ ]:
battles = pd.DataFrame([json.loads(line) for line in open("../datasets/battle-features-1.json")])
battles = battles[battles.start_date.notnull()]
battles.fillna(value="None", inplace=True)
def get_year(date):
    return datetime.strptime(date, "%Y-%m-%d").year if date else None

def get_duration(battle):
    if battle.end_date == "None":
        return 1
    else:
        return (datetime.strptime(battle.end_date, "%Y-%m-%d") - datetime.strptime(battle.start_date, "%Y-%m-%d")).days

battles["year"] = battles["start_date"].apply(get_year)

battles = battles[(battles.year>=1000) & (battles.year <= 2018)]

battles["duration"] = [get_duration(battle) for _, battle in battles.iterrows()]
decades = range(int(battles.year.min()), int(battles.year.max())+10, 10)
centuries = range(int(battles.year.min()), int(battles.year.max())+100, 100)
battles["decade"] = pd.cut(battles["year"], decades)
battles["century"] = pd.cut(battles["year"], centuries)

In [ ]:
b = battles.decade.value_counts()
fig, ax = plt.subplots(figsize=(10,16))
sns.barplot(y=b.index, x=b, color="blue",ax=ax)

In [ ]:
fig, (ax1,ax2) = plt.subplots(1, 2, figsize=(20,10))
sns.set(color_codes=True)

b = battles[battles.duration.between(*battles.duration.quantile([.25, .75]))]

bdec = b.groupby('decade').duration.mean()
bcent = b.groupby('century').duration.mean()

sns.regplot(x=np.array(decades[:-1]), y=np.array(bdec), order=4, ax=ax1)
sns.regplot(x=np.array(centuries[:-1]), y=np.array(bcent), order=4, ax=ax2)
#battles[["title", "start_date", "end_date", "duration"]].sort_values(by="duration", ascending=False)

In [ ]:
battles.duration.between(*battles.duration.quantile([.2, .8])).sum()